# Generate offset string in hal setting file, and corresponding shutter file

By Pu Zheng

2021.5.31

In [1]:
# import required packages
import os, sys, time
import numpy as np

# generate the offset string

paste the output string into 
</locked>
  	<hardware_z_scan>
		<z_offsets type="string">

In [26]:
# required parameters
num_steps = 60
step_size = 0.2
num_channels = 2

In [27]:
offset_string = []
start_z = 0
for _i in np.arange(num_steps):
    offset_string.extend([str(round(start_z+_i*step_size, 2))]*num_channels)

In [28]:
# copy the output of this box into the hal setting:
print(','.join(offset_string))

0.0,0.0,0.2,0.2,0.4,0.4,0.6,0.6,0.8,0.8,1.0,1.0,1.2,1.2,1.4,1.4,1.6,1.6,1.8,1.8,2.0,2.0,2.2,2.2,2.4,2.4,2.6,2.6,2.8,2.8,3.0,3.0,3.2,3.2,3.4,3.4,3.6,3.6,3.8,3.8,4.0,4.0,4.2,4.2,4.4,4.4,4.6,4.6,4.8,4.8,5.0,5.0,5.2,5.2,5.4,5.4,5.6,5.6,5.8,5.8,6.0,6.0,6.2,6.2,6.4,6.4,6.6,6.6,6.8,6.8,7.0,7.0,7.2,7.2,7.4,7.4,7.6,7.6,7.8,7.8,8.0,8.0,8.2,8.2,8.4,8.4,8.6,8.6,8.8,8.8,9.0,9.0,9.2,9.2,9.4,9.4,9.6,9.6,9.8,9.8,10.0,10.0,10.2,10.2,10.4,10.4,10.6,10.6,10.8,10.8,11.0,11.0,11.2,11.2,11.4,11.4,11.6,11.6,11.8,11.8


# generate the shutter file

In [29]:
all_channels= [750, 647, 561, 488, 405]

#ch_ids = [0, 1, 3, 4]
#ch_ids = [0, 1, 2, 3]
#ch_ids = [0, 1, 3]
#ch_ids = [1, 3, 4]
ch_ids = [1, 3]

In [30]:
full_shutter_str = ""

## header
header = f"""<?xml version="1.0" encoding="ISO-8859-1"?>
<repeat>
  <oversampling>1</oversampling>
  <frames>{num_steps*num_channels}</frames>
  """
# append header
full_shutter_str += header
## loop through channels
for _i  in np.arange(num_steps):
    full_shutter_str += """
  """
    for _j, _ch in enumerate(ch_ids[:num_channels]):
        # determine start time and end time
        _start_time = _i * num_channels + _j
        _end_time = _i * num_channels + _j + 1
        # determine _channel string
        if _ch < 0:
            continue
        else:
            _ch_str = f"""<event>
    <channel>{_ch}</channel>
    <power>1</power>
    <on>{_start_time:.1f}</on>
    <off>{_end_time:.1f}</off>
    <color>255,255,255</color>
  </event>
  """
        # append
        full_shutter_str += _ch_str

# append the tail
full_shutter_str += """
</repeat>"""

In [31]:
print(full_shutter_str)

<?xml version="1.0" encoding="ISO-8859-1"?>
<repeat>
  <oversampling>1</oversampling>
  <frames>120</frames>
  
  <event>
    <channel>1</channel>
    <power>1</power>
    <on>0.0</on>
    <off>1.0</off>
    <color>255,255,255</color>
  </event>
  <event>
    <channel>3</channel>
    <power>1</power>
    <on>1.0</on>
    <off>2.0</off>
    <color>255,255,255</color>
  </event>
  
  <event>
    <channel>1</channel>
    <power>1</power>
    <on>2.0</on>
    <off>3.0</off>
    <color>255,255,255</color>
  </event>
  <event>
    <channel>3</channel>
    <power>1</power>
    <on>3.0</on>
    <off>4.0</off>
    <color>255,255,255</color>
  </event>
  
  <event>
    <channel>1</channel>
    <power>1</power>
    <on>4.0</on>
    <off>5.0</off>
    <color>255,255,255</color>
  </event>
  <event>
    <channel>3</channel>
    <power>1</power>
    <on>5.0</on>
    <off>6.0</off>
    <color>255,255,255</color>
  </event>
  
  <event>
    <channel>1</channel>
    <power>1</power>
    <on>6.0</on>
  

## save this shutter file

In [32]:
save_folder = r'C:\Users\neoSTORM6\Documents\STORM6\example_scripts\hal'
save_folder = os.path.join(save_folder, f'{num_steps}_steps')
save_basename = "shutter_" + '_'.join([str(all_channels[_id]) for _id in ch_ids[:num_channels]]) + f'_s{num_steps}.xml'
save_filename = os.path.join(save_folder, save_basename)
overwrite = False
if not os.path.isfile(save_filename) or overwrite:
    print(f"-- save the shutter to file: {save_filename}")
    # write
    with open(save_filename, 'w') as _fp:
        _fp.write(full_shutter_str)
else:
    print(f"-- shutter file: {save_filename} already exists, skip")

-- save the shutter to file: C:\Users\neoSTORM6\Documents\STORM6\example_scripts\hal\60_steps\shutter_647_488_s60.xml
